# RDFSolve: PubChem Endpoint Analysis

This notebook analyzes the PubChem Endpoint graph using RDFSolve:
- **Graph URI**: http://rdf.ncbi.nlm.nih.gov/pubchem/endpoint
- **SPARQL Endpoint**: https://idsm.elixir-czech.cz/sparql/endpoint/idsm
- **Dataset**: PubChem Endpoint metadata

Explore the structure and schema of the PubChem Endpoint dataset.

In [ ]:
import pandas as pd
from rdfsolve.rdfsolve import RDFSolver
from rdfsolve.void_parser import VoidParser
import warnings
warnings.filterwarnings('ignore')

## Step 1: Configure Dataset Parameters

In [ ]:
# PubChem Endpoint configuration
endpoint_url = "https://idsm.elixir-czech.cz/sparql/endpoint/idsm"
graph_uri = "http://rdf.ncbi.nlm.nih.gov/pubchem/endpoint"
void_iri = "http://rdf.ncbi.nlm.nih.gov/pubchem/endpoint"
dataset_name = "pubchem_endpoint"
working_path = "."

print(f"Dataset: {dataset_name}")
print(f"Endpoint: {endpoint_url}")
print(f"Graph URI: {graph_uri}")
print(f"VoID IRI: {void_iri}")

## Step 2: Initialize RDFSolver

In [ ]:
try:
    solver = RDFSolver(
        endpoint=endpoint_url,
        path=working_path,
        void_iri=void_iri,
        dataset_name=dataset_name
    )
    
    print("RDFSolver initialized successfully")
    print(f"Endpoint: {solver.endpoint}")
    print(f"Dataset: {solver.dataset_name}")
    
except Exception as e:
    print(f"Error: {e}")

## Step 3: Generate VoID Description

In [ ]:
try:
    print("Generating VoID description...")
    
    void_graph = solver.void_generator(
        graph_uri=graph_uri,
        output_file=f"{dataset_name}_void.ttl",
        counts=True
    )
    
    print(f"VoID generation completed!")
    print(f"Graph contains {len(void_graph)} triples")
    print(f"Saved to: {dataset_name}_void.ttl")
    
except Exception as e:
    print(f"VoID generation failed: {e}")

## Step 4: Extract Schema

In [ ]:
try:
    print("Extracting schema from VoID...")
    parser = VoidParser(void_graph)
    
    schema_df = parser.to_schema(filter_void_nodes=True)
    
    print("Schema extraction completed")
    print(f"Total schema triples: {len(schema_df)}")
    print(f"Unique classes: {schema_df['subject_class'].nunique()}")
    print(f"Unique properties: {schema_df['property'].nunique()}")
    
except Exception as e:
    print(f"Schema extraction failed: {e}")

## Step 5: Schema Visualization

In [ ]:
# Display schema sample
if 'schema_df' in locals():
    print("Schema Sample (first 10 rows):")
    display(schema_df.head(10))
    
    print("\nTop 10 Classes by Property Count:")
    class_counts = schema_df['subject_class'].value_counts().head(10)
    for cls, count in class_counts.items():
        print(f"  {cls}: {count} properties")

## Step 6: Domain-Specific Analysis

#TODO: Add PubChem Endpoint-specific analysis

In [ ]:
# TODO: Implement pubchem endpoint-specific analysis
print("TODO: Add pubchem endpoint analysis")

## Step 7: Export Results

In [ ]:
try:
    if 'parser' in locals():
        # Export as JSON
        schema_json = parser.to_json(filter_void_nodes=True)
        
        import json
        with open(f"{dataset_name}_schema.json", "w") as f:
            json.dump(schema_json, f, indent=2)
        
        # Export as CSV
        schema_df.to_csv(f"{dataset_name}_schema.csv", index=False)
        
        print(f"Results exported:")
        print(f"  - {dataset_name}_void.ttl")
        print(f"  - {dataset_name}_schema.json")
        print(f"  - {dataset_name}_schema.csv")
        
except Exception as e:
    print(f"Export failed: {e}")